# Read contents of .nwb file

In [1]:
from pynwb import NWBHDF5IO

# Replace with the path to your .nwb file
nwb_file_path = 'data1.nwb'


# Open the .nwb file using PyNWB
with NWBHDF5IO(nwb_file_path, 'r') as io:
    nwbfile = io.read()

    # Access the acquisition group
    acquisition = nwbfile.acquisition

    # Print the names of all groups within the acquisition group
    print("Acquisition Groups:")
    for name, timeseries in acquisition.items():
        print(f" - {name}: {timeseries}")
    
    # Print all groups in stimulus (if any)
    if hasattr(nwbfile, 'stimulus'):
        print("\nStimulus Groups:")
        for name, sgroup in nwbfile.stimulus.items():
            print(f" - {name}: {sgroup}")

Acquisition Groups:
 - ElectricalSeries: ElectricalSeries pynwb.ecephys.ElectricalSeries at 0x139943538298736
Fields:
  comments: voltage data recorded from the amplifiers of an Intan Technologies chip
  conversion: 1.0
  data: <HDF5 dataset "data": shape (386048, 128), type "<f4">
  description: voltage data recorded from the amplifiers of an Intan Technologies chip
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  filtering: Wideband data
  interval: 1
  offset: 0.0
  resolution: 1.95e-07
  timestamp_link: (
    TimeSeries_analog_output <class 'pynwb.base.TimeSeries'>,
    TimeSeries_digital_input <class 'pynwb.base.TimeSeries'>,
    TimeSeries_amp_settle <class 'pynwb.base.TimeSeries'>,
    TimeSeries_charge_recovery <class 'pynwb.base.TimeSeries'>,
    TimeSeries_compliance_limit <class 'pynwb.base.TimeSeries'>,
    TimeSeries_stimulation <class 'pynwb.base.TimeSeries'>
  )
  timestamps: <HDF5 dataset "timestamps": shape (386048,), type "<f8">
  timestamps_u

# Extract nwb file contents

In [2]:
import numpy as np
from pynwb import NWBHDF5IO

# Function to extract data from a TimeSeries object
def extract_data(time_series):
    return {
        'data': np.array(time_series.data[:]),
        'timestamps': np.array(time_series.timestamps[:]) if time_series.timestamps else None,
        'unit': time_series.unit,
        'comments': time_series.comments
    }

# Path to your NWB file
nwb_file_path = 'data1.nwb'

# Open the .nwb file using PyNWB
with NWBHDF5IO(nwb_file_path, 'r') as io:
    nwbfile = io.read()

    # Extract acquisition groups
    acquisition_data = {}
    for name, timeseries in nwbfile.acquisition.items():
        acquisition_data[name] = extract_data(timeseries)

    # Extract stimulus groups if present
    stimulus_data = {}
    if hasattr(nwbfile, 'stimulus'):
        for name, timeseries in nwbfile.stimulus.items():
            stimulus_data[name] = extract_data(timeseries)

# Save extracted contents of nwb file as numpy

In [ ]:
for name, data in acquisition_data.items():
    np.save(f'{name}_acquisition.npy', data['data'])

for name, data in stimulus_data.items():
    np.save(f'{name}_stimulus.npy', data['data'])

# Save extracted contents of nwb file as dataframe csv

In [10]:
import pandas as pd
import numpy as np

def save_to_csv(data_dict, file_suffix):
    for name, data in data_dict.items():
        # Create column names based on the shape of the data
        column_names = [f'{name}_{i}' for i in range(data['data'].shape[1])]
        
        # Create a DataFrame
        df = pd.DataFrame(data['data'], columns=column_names)
        
        # Drop columns where all values are zero
        df = df.loc[:, (df != 0).any(axis=0)]
        
        # Add timestamps if available
        if data['timestamps'] is not None:
            df.insert(0, 'timestamps', data['timestamps'])
        
        # Save DataFrame to CSV
        csv_filename = f'{name}_{file_suffix}.csv'
        df.to_csv(csv_filename, index=False)
        print(f'Saved: {csv_filename}')

# Assuming acquisition_data and stimulus_data are dictionaries 
# containing your extracted data and timestamps:
save_to_csv(acquisition_data, 'acquisition')
save_to_csv(stimulus_data, 'stimulus')


Saved: ElectricalSeries_acquisition.csv
Saved: TimeSeries_analog_output_acquisition.csv
Saved: TimeSeries_digital_input_acquisition.csv
Saved: TimeSeries_amp_settle_stimulus.csv
Saved: TimeSeries_charge_recovery_stimulus.csv
Saved: TimeSeries_compliance_limit_stimulus.csv
Saved: TimeSeries_stimulation_stimulus.csv
